<a href="https://colab.research.google.com/github/Vixuz144/Simulacion-1/blob/main/Guia_segundo_parcial_Figols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from random import random
from numba import jit
import matplotlib.pyplot as plt

#1

In [2]:
@jit(nopython = True)
def le_1s(T, ll, ls, rep):
  lt, li = [], []
  lm, lp = [], []
  for k in range(rep):
    t, i = 0, 0
    ltl, lts = [], []

    # lista_tiempos = [t]
    # lista_individuos = [i]

    tl = t - np.log(random())/ll
    ts = np.inf
    t = min(tl,ts)
    mx = 0

    while t < T:
      mx = max(mx, i)
      if t == tl: #Llegada
        i += 1
        ltl.append(tl)
        tl = t - np.log(random())/ll
        if i == 1: #si es el primer cliente
          ts = t - np.log(random())/ls
      else: #Salida
        i -= 1
        lts.append(ts)
        if i == 0: #Se vacia
          ts = np.inf
        else:
          ts = t - np.log(random())/ls
      t = min(tl,ts)
      # lista_tiempos.append(t)
      # lista_individuos.append(i)

    n = len(lts)
    if n < len(ltl):
      ltl = ltl[:n]
    tp = np.array(lts) - np.array(ltl)

    li.append(i)
    lt.append(t)
    lm.append(mx)
    lp.append(tp.mean())
  return lt, li, lm, lp

In [3]:
ll = 2
ls = 2.1
T = 60
nr = 10**5
lt, li, lm, lp = le_1s(T, ll, ls, nr)

print('El valor esperado del número medio de usuarios dentro del sistema es:',np.mean(li))
print('El valor esperado del máximo número de personas dentro del sistema es:',np.mean(lm))
print('El valor esperado del del tiempo promedio de permanencia de los usuario dentro del sistema es:',np.mean(lp))

El valor esperado del número medio de usuarios dentro del sistema es: 9.35164
El valor esperado del máximo número de personas dentro del sistema es: 16.37689
El valor esperado del del tiempo promedio de permanencia de los usuario dentro del sistema es: 3.1924471775523764


#2

In [4]:
# @jit(nopython = True)
def le_ns(T,ll,ls, N = 10**5):
  li = []
  lm = []
  lp = []
  for i in range(N):
    n = len(list(ls))
    t = 0
    i = 0

    # l_t = [t]
    # l_i = [i]
    # l_k = [0]
    # l_c = [0]

    lll, sss = [], []

    mx = 0

    tl = 0
    ts = list(np.inf * np.ones(n))

    cola = 0

    while min(tl, min(ts)) < T:
      if t == tl: #llegada
        i += 1
        tl = t - np.log(random())/ll
        lll.append(tl)

        if np.any(np.isinf(ts)): #Hay servidores vacios
          lk = []
          for k in range(n):
            if np.isinf(ts[k]):
              lk.append(k)

          m = len(lk)
          l = np.linspace(0,1,m+1)
          l[-1] += 0.1
          r = random()

          for j in range(m):
            a, b = l[j], l[j+1]
            if a <= r < b :
              k = lk[j]
              ts[k] = t - np.log(random())/ls[k]

        else: #No hay servidores vacios
          cola += 1

      else: #salida
        i -= 1
        k = list(ts).index(min(ts))
        sss.append(ts[k])

        if i < n:
          ts[k] = np.inf
        elif cola >= 1 :
          ts[k] = t - np.log(random())/ls[k]
          cola -= 1

      t = min(tl, min(ts))
      mx = max(mx, i)
      # l_t.append(t)
      # l_i.append(i)
      # l_k.append(k)
      # l_c.append(cola)
    n = len(sss)
    if n < len(lll):
      lll = lll[:n]
    tp = np.array(sss) - np.array(lll)

    li.append(i)
    lm.append(mx)
    lp.append(tp.mean())
  return li, lm, lp
  #   l_c = np.array(l_c[:-1])
  #   lm.append(l_c.mean())
  # lm = np.array(lm)
  # return lm.mean()

In [5]:
ll = 2
ls = [0.7, 0.7, 0.7]
T = 60
nr = 10**4

li, lm, lp = le_ns(T, ll, ls, nr)

print('El valor esperado del número medio de usuarios dentro del sistema es:',np.mean(li))
print('El valor esperado del máximo número de personas dentro del sistema es:',np.mean(lm))
print('El valor esperado del del tiempo promedio de permanencia de los usuario dentro del sistema es:',np.mean(lp))

El valor esperado del número medio de usuarios dentro del sistema es: 10.2717
El valor esperado del máximo número de personas dentro del sistema es: 17.3754
El valor esperado del del tiempo promedio de permanencia de los usuario dentro del sistema es: 3.161070190205058


#3

In [6]:
def F(alpha):
  return  -np.log(random())/alpha

In [7]:
from types import MemberDescriptorType
def rep_maq(p_desc = 1/50, p_rep = 1/5, res = 3, nm = 5, rep = 10**4):
  lt = []
  ld = []
  for i in range(rep):
    t = 0
    met = 0 # máquinas que necesitan reparación, máquinas en taller
    disp = res # almacen de máquinas de repuesto disponibles para remplazar alguna que necesita reparación

    d = 0

    td = [] #Lista de máquinas que se pueden descomponer
    for i in range(nm):
      td.append(-np.log(random())/p_desc)
    tdp = min(td) # tiempo más próximo en que se descompondrá una máquina
    tsp = np.inf # Riempo de reparación de la máquina en taller

    indicador = True # hay máquinas disponibles de remplazo
    while indicador :
      t=min(tdp,tsp)

      if t == tdp: #EVENTO: se descompone una máquina
        d += 1
        if disp==0:# ya no hay máquinas disponibles para remplazo
          indicador = False
          k = td.index(tdp)
        else: #hay máquinas disponibles para remplazo
          disp -= 1
          met += 1
          k = td.index(tdp)
          td[k] = t - np.log(random())/p_desc
          if met == 1:
            tsp = t - np.log(random())/p_rep
          tdp = min(td)
        # texto = 'Descomp'
      else: #EVENTO: se repara una máquina descompuesta
        met -= 1
        disp += 1
        if met == 0:
          tsp = np.inf
        else:
          tsp = t - np.log(random())/p_rep
        # texto = 'Fin Rep'
      # print(texto, round(t,5), k, disp, MemberDescriptorType)
    lt.append(t)
    ld.append(d)
  return lt, ld

In [8]:
lt, ld = rep_maq(rep = 10**3)
print(f"Valor esperado del tiempo de la primera interrupción: {np.mean(lt)}")
print(f'Valor esperado del número de descomposturas antes de la primera interrupción: {np.mean(ld)}')

Valor esperado del tiempo de la primera interrupción: 259.10057909424467
Valor esperado del número de descomposturas antes de la primera interrupción: 25.634


#4

In [9]:
@jit(nopython = True)
def inventario(nr = 10**4):
  g = []
  vacio = []
  for i in range(nr):
    # cap = cap0
    cap = 100
    ll = 5
    T = 365

    cant_compra = 1
    precio = 2
    costo_a = 0.05

    tr = 15
    periodicidad = 15

    cap_min = 40
    cap_max = 100

    t = 0
    v = 0
    vp = 0
    tp = 0

    ganancia = 0
    venta = 0
    costo = 0
    tv = -np.log(random())/ll

    while t < T:
      ta = t
      t = min(tv,tr)
      costo += (t-ta) * cap * costo_a
      if t==tv:
        if cap > 0:
          v += cant_compra
          cap -= cant_compra
          venta += cant_compra * precio
        else:
          vp += cant_compra
          tp = tp + (t - ta)
        tv = tv - np.log(random())/ll
      else: # t == tr
        if cap < cap_min:
          cap = cap_max
        tr = tr + periodicidad
    ganancia = venta - costo
    g.append(ganancia)
    vacio.append(tp)
  return g, vacio

In [10]:
g, tva = inventario(10**5)
print('Ganancia esperada:',np.mean(g))
print("Tiempo promedio de almacen vacio:",np.mean(tva))


Ganancia esperada: 2478.649037824275
Tiempo promedio de almacen vacio: 5.985855896205094
